In [2]:
import sys
import pandas as pd
sys.path.append("../")

import utils
import numpy as np

In [3]:
data = utils.load_sdd_client_data("../data3.json")

In [4]:
df = pd.DataFrame(data)

In [20]:
df.index

RangeIndex(start=0, stop=1693, step=1)

In [9]:
df["time_diff"] = np.abs(df["packageTime"]*1000-df["cameraTime"])

In [19]:
df.groupby("packageIndex")["time_diff"].min().index

IndexError: positional indexers are out-of-bounds

In [39]:
ret_df = []
for index, diff in df.groupby("packageIndex")["time_diff"].min().items():
    ret_df.append(df[df["packageIndex"] == index][df["time_diff"] == diff])

/home/nrx/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [42]:
table = pd.concat(ret_df)

In [47]:
table["pointOx"].unique()

array([  0, 243, 238, 236, 235, 237, 233, 232, 229, 226, 215, 209, 200,
       199, 196, 180, 177, 176, 163, 159, 142, 137, 124, 120, 105, 101,
        87,  81,  64,  61,  46,  43,  29,  27,  24,  22,  21,  18,  19,
        26,  30,  32,  34,  38,  40,  41,  44,  45,  49,  50,  51,  53,
        55,  57,  62,  66,  73,  76,  78,  86,  92,  95,  96, 102, 104,
       110, 113, 116, 125, 130, 139, 140, 149, 154, 156, 157, 160, 164,
       165, 167, 170, 171, 172, 178, 184, 186, 189, 191, 197, 204, 207,
       212, 214, 217, 221, 223, 230, 239, 242, 247, 250, 252, 257, 260,
       263, 269, 271, 274, 279, 283, 287, 289, 292, 298, 301, 303, 305,
       306, 307, 309, 302, 300, 296, 294, 288, 284, 282, 280, 278, 276,
       268, 264, 261, 259, 254, 246, 244, 234, 225, 222, 210, 205, 195,
       190, 188, 182, 173, 161, 152, 147, 135, 133, 123, 118, 115, 108,
        93,  91,  84,  75,  72,  48,  47,  42,  54,  59,  67,  70,  79,
        82,  90,  94, 100, 111, 117, 129, 131, 143, 145, 151, 15

In [74]:
table

,packageIndex,ox,oy,pwmX,pwmY,taskX,taskY,packageTime,pointOx,pointOy,cameraTime,cameraIndex,time_diff
0,3080592,-169,-169,255,255,0,0,26430778,0,0,26431406710,3692,628710
15,3080593,169,169,-255,-255,0,0,26431426,0,0,26431425876,3707,124
19,3080594,159,159,255,255,0,0,26431537,0,0,26431535462,3711,1538
22,3080595,129,129,255,255,0,0,26431586,0,0,26431607330,3714,21330
24,3080596,78,78,255,255,0,0,26431646,0,0,26431644461,3716,1539
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1682,3081294,159,159,255,255,0,0,26470138,0,0,26470129459,5374,8541
1685,3081295,129,129,255,255,0,0,26470193,0,0,26470205303,5377,12303
1687,3081296,78,78,255,255,0,0,26470251,0,0,26470249578,5379,1422
1689,3081297,4,4,255,255,0,0,26470306,0,0,26470288172,5381,17828


In [79]:
width = 320
height = 240
window_widh = 0.94   # ширина записывыемого полотна
window_height = 0.94 # высота записываемого полотна
center = (width/2, height/2)
Px = 0.94 / width  # ширина пиксиля в метрах
Py = 0.94 / height # высота писеля в метрах
ox = ((center[0] - table["pointOx"]) * Px).to_numpy()
oy = ((center[1] - table["pointOy"]) * Py).to_numpy()
oz = pd.Series([1.88]*ox.shape[0]).to_numpy()

In [97]:
table["pointOy"].unique()

array([  0,  46,  53,  65,  64,  62,  60,  61,  59,  58,  63,  69,  67,
        68,  56,  55,  54,  51,  50,  49,  48,  52,  57,  45,  44,  43,
        41,  40,  39,  42,  47,  38,  37,  36,  35,  34,  32,  31,  30,
        28,  27,  26,  24,  23,  22,  21,  25,  29,  19,  18,  73,  77,
        81,  88,  91,  92,  85,  15,   6,   8,  16,  70,  90,  97, 104,
       105, 102,  98,  75,  33,  13,  12,  66,  74,  93, 100, 109, 108,
        14,   2,  96,  94,   4,  10,  72,  20,  89,  86, 224, 225, 221,
       222])

In [73]:
table["ox"]

0      -169
15      169
19      159
22      129
24       78
       ... 
1682    159
1685    129
1687     78
1689      4
1692    -92
Name: ox, Length: 707, dtype: int64

In [85]:
# использование сферической системы координат не верно!
r = np.sqrt(np.power(ox, 2) + np.power(oy, 2) + np.power(oz, 2))
xDeg = np.arctan(np.sqrt(np.power(ox, 2) + np.power(oy, 2))/oz)
yDeg = np.arctan(oy/ox)

/home/nrx/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [91]:
xDeg = np.arctan(ox/oz)
yDeg = np.arctan(oy/oz)

In [94]:
xGrad = xDeg*180/np.pi
yGrad = yDeg*180/np.pi
xGrad, yGrad

(array([ 14.03624347,  14.03624347,  14.03624347,  14.03624347,
         14.03624347,  14.03624347,  14.03624347,  14.03624347,
         14.03624347,  14.03624347,  14.03624347,  14.03624347,
         14.03624347,  14.03624347,  14.03624347,  14.03624347,
         14.03624347,  14.03624347,  14.03624347,  14.03624347,
         14.03624347,  14.03624347,  14.03624347,  14.03624347,
         14.03624347,  14.03624347,  14.03624347,  14.03624347,
         14.03624347,  14.03624347,  14.03624347,  14.03624347,
         14.03624347,  14.03624347,  14.03624347,  14.03624347,
         14.03624347,  14.03624347,  14.03624347,  14.03624347,
         14.03624347,  14.03624347,  14.03624347,  14.03624347,
         14.03624347,  14.03624347,  14.03624347,  14.03624347,
         14.03624347,  14.03624347,  14.03624347,  14.03624347,
         14.03624347,  14.03624347,  14.03624347,  14.03624347,
         14.03624347,  14.03624347,  14.03624347,  14.03624347,
         14.03624347,  14.03624347,  14.

In [95]:
oy

array([ 0.47      ,  0.47      ,  0.47      ,  0.47      ,  0.47      ,
        0.47      ,  0.47      ,  0.47      ,  0.47      ,  0.47      ,
        0.47      ,  0.47      ,  0.47      ,  0.47      ,  0.47      ,
        0.47      ,  0.47      ,  0.47      ,  0.47      ,  0.47      ,
        0.47      ,  0.47      ,  0.47      ,  0.47      ,  0.47      ,
        0.47      ,  0.47      ,  0.47      ,  0.47      ,  0.47      ,
        0.47      ,  0.47      ,  0.47      ,  0.47      ,  0.47      ,
        0.47      ,  0.47      ,  0.47      ,  0.47      ,  0.47      ,
        0.47      ,  0.47      ,  0.47      ,  0.47      ,  0.47      ,
        0.47      ,  0.47      ,  0.47      ,  0.47      ,  0.47      ,
        0.47      ,  0.47      ,  0.47      ,  0.47      ,  0.47      ,
        0.47      ,  0.47      ,  0.47      ,  0.47      ,  0.47      ,
        0.47      ,  0.47      ,  0.47      ,  0.47      ,  0.47      ,
        0.47      ,  0.47      ,  0.47      ,  0.47      ,  0.47